In [ ]:
%run optimizers/Optimizer.ipynb
%run optimizers/PoseEstimation.ipynb
%run optimizers/ShapeEstimation.ipynb

class AlternatingPoseShapeEstimation2(Optimizer):
    
    def __init__(self, mean, pcaBasis, pcaVariance, numIterations=3):
        Optimizer.__init__(self, mean, pcaBasis, pcaVariance)
        self.numIterations = numIterations
        self.ai, self.ri, self.ti, self.si = Optimizer.getDefaults(self.numComponents)
        self.poseEstimator = PoseEstimation(mean, pcaBasis, pcaVariance, 2000)
        self.shapeEstimator = ShapeEstimation(mean, pcaBasis, pcaVariance, 10000)
    
    def open(self):
        self.poseEstimator.open()
        self.shapeEstimator.open()
    
    def close(self):
        self.poseEstimator.close()
        self.shapeEstimator.close()
    
    def optimize(self, targetFeatures):
        for _ in range(self.numIterations):
            self.ai, self.ri, self.ti, self.si = self.poseEstimator.optimize(targetFeatures)
            self.shapeEstimator.update(self.ai, self.ri, self.ti, self.si)
            self.ai, self.ri, self.ti, self.si = self.shapeEstimator.optimize(targetFeatures)
            self.poseEstimator.update(self.ai, self.ri, self.ti, self.si)
        return self.ai, self.ri, self.ti, self.si
    
    def update(self, a, r, t, s):
        self.poseEstimator.update(a, r, t, s)
        self.shapeEstimator.update(a, r, t, s)
    